# Image Recognition (PART A)

### Import libraries

In [ ]:
import numpy as np
import pandas as pd

import sklearn.linear_model
import sklearn.pipeline
from sklearn.utils import shuffle

# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

from sklearn.neural_network import MLPClassifier
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

### Load Training Data

In [ ]:
import os

data_dir = os.path.abspath("data_fashion/")
# Load data
train_x = pd.read_csv(os.path.join(data_dir, "x_train.csv")).to_numpy()
train_y_df = pd.read_csv(os.path.join(data_dir, "y_train.csv"))
valid_x = pd.read_csv(os.path.join(data_dir, "x_valid.csv")).to_numpy()
valid_y_df = pd.read_csv(os.path.join(data_dir, "y_valid.csv"))
test_x = pd.read_csv(os.path.join(data_dir, "x_test.csv")).to_numpy()

# Print shapes
for label, arr in [('train', train_x), ('valid', valid_x)]:
    print("Contents of %s_x.csv: arr of shape %s" % (
        label, str(arr.shape)))

for label, arr in [('train', train_y_df), ('valid', valid_y_df)]:
    print("Contents of %s_x.csv: arr of shape %s" % (
        label, str(arr.shape)))

print("Contents of test_x.csv: arr of shape %s \n" % (
         str(arr.shape)))

class_counts_train = train_y_df['class_name'].value_counts()
class_counts_valid = valid_y_df['class_name'].value_counts()

print("Class Counts: training data")
print(class_counts_train)
print('\n')
print("Class Counts: validation data")
print(class_counts_valid)

Contents of train_x.csv: arr of shape (2102, 784)
Contents of valid_x.csv: arr of shape (600, 784)
Contents of train_x.csv: arr of shape (2102, 2)
Contents of valid_x.csv: arr of shape (600, 2)
Contents of test_x.csv: arr of shape (600, 2) 

Class Counts: training data
sandal      800
sneaker     800
dress       400
pullover    100
top           1
trouser       1
Name: class_name, dtype: int64


Class Counts: validation data
dress       100
trouser     100
sandal      100
top         100
pullover    100
sneaker     100
Name: class_name, dtype: int64


### Pack Training and validation sets into big arrays (for sklearn hyperparameter search)

In [ ]:
xall_LF = np.vstack([train_x, valid_x])
yall_L = np.vstack([train_y_df[['class_name']], valid_y_df[['class_name']]])

print(xall_LF.shape)
print(yall_L.shape)

(2702, 784)
(2702, 1)


In [ ]:
valid_indicators_L = np.hstack([
    -1 * np.ones(train_y_df[['class_name']].size),
    0 * np.ones(valid_y_df[['class_name']].size)
])

valid_indicators_L

In [ ]:
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

### Perform Grid Search

In [ ]:
# Parameter grid
param_grid = {
    'alpha': np.logspace(-6, 6, 12),
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [64, 128, 256, 512]
}

In [ ]:
# Create a mlp model for classification
clf = MLPClassifier(activation='relu', hidden_layer_sizes=100, max_iter=1000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy',
                           cv=my_splitter, refit=False, return_train_score=True)

#Run Grid Search
grid_search.fit(xall_LF, yall_L.ravel())

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
cv_results_df = pd.DataFrame(grid_search.cv_results_)
print(cv_results_df)

file_path = "cv_results.csv"
cv_results_df.to_csv(file_path, index=False)

### Confusion Matrix


In [ ]:
# Get best model from grid search
mlp_best = MLPClassifier(activation='relu', hidden_layer_sizes=100, max_iter=1000,
                         alpha=1e-06, batch_size=64, learning_rate='constant')

mlp_best.fit(train_x, train_y_df['class_name'].values)

# Get predictions from validation set
yhat_valid = mlp_best.predict(valid_x)

# Convert valid data to numpy array
y_valid = valid_y_df['class_name'].values

print(y_valid.shape)
print(yhat_valid.shape)

In [ ]:
# Create confusion matrix
confusion_matrix(y_valid, yhat_valid, labels=["dress", "pullover", "top", "trouser", "sandal", "sneaker"])

### Training Set Modification


In [ ]:
# Contents of train_x.csv: arr of shape (2102, 784)

'''
Contents of valid_x.csv: arr of shape (600, 2)
sandal      800
sneaker     800
dress       400
pullover    100
top           1
trouser       1
'''
y_train = train_y_df['class_name'].values
modified_x_train = []
modified_y_train = []

for i in range(2102):
    if(y_train[i] == "pullover"):
        for a in range(8):
            modified_x_train.append(train_x[i])
            modified_y_train.append(y_train[i])

    if (y_train[i] == "dress"):
        for b in range(2):
            modified_x_train.append(train_x[i])
            modified_y_train.append(y_train[i])

    if((y_train[i] == "top") or (y_train[i] == "trouser")):
        for c in range(800):
            modified_x_train.append(train_x[i])
            modified_y_train.append(y_train[i])

    if((y_train[i] == "sandal") or (y_train[i] == "sneaker")):
        modified_x_train.append(train_x[i])
        modified_y_train.append(y_train[i])



print(len(modified_x_train))
print(len(modified_y_train))

modified_y_train_df = pd.DataFrame(modified_y_train, columns=['class_name'])

4800
4800


### Pack Training and validation sets into big arrays (for sklearn hyperparameter search)

In [ ]:
xall_LF_2 = np.vstack([modified_x_train, valid_x])
yall_L_2 = np.vstack([modified_y_train_df, valid_y_df[['class_name']]])

print(xall_LF_2.shape)
print(yall_L_2.shape)



(5400, 784)
(5400, 1)


In [ ]:
valid_indicators_L_2 = np.hstack([
    -1 * np.ones(modified_y_train_df.size),
    0 * np.ones(valid_y_df[['class_name']].size)
])

valid_indicators_L_2

In [ ]:
my_splitter_2 = sklearn.model_selection.PredefinedSplit(valid_indicators_L_2)

In [ ]:
#parameter grid
param_grid_2 = {
    'activation': ['logistic', 'relu'],
    'alpha': np.logspace(-6, 6, 12),
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [64, 128, 256, 512]
}

In [ ]:
# Create a mlp model for classification
clf_2 = MLPClassifier(hidden_layer_sizes=100, max_iter=1000)

grid_search_2 = GridSearchCV(clf_2, param_grid_2, scoring='balanced_accuracy',
                           cv=my_splitter_2, refit=False, return_train_score=True)

#Run Grid Search
grid_search_2.fit(xall_LF_2, yall_L_2)

In [ ]:
grid_search_2.best_score_

In [ ]:
grid_search_2.best_params_

In [ ]:
# Get best model from grid search
mlp_best_2 =  MLPClassifier(activation='relu', hidden_layer_sizes=100, max_iter=1000,
                         alpha=533.6699231206302, batch_size=64, learning_rate='adaptive')

mlp_best_2.fit(modified_x_train, modified_y_train)

# Get predictions from validation set
yhat_valid_2 = mlp_best_2.predict(valid_x)

# Convert valid data to numpy array
y_valid = valid_y_df['class_name'].values

print(y_valid.shape)
print(yhat_valid.shape)


(600,)
(600,)


In [ ]:
confusion_matrix(y_valid, yhat_valid_2, labels=["dress", "pullover", "top", "trouser", "sandal", "sneaker"])

### Performance on Test Set via Leaderboard


In [ ]:
yhat_test = mlp_best_2.predict(test_x)

# Define the file path where you want to save the CSV file
# file_path = 'yproba1_test.txt'

# Save the 'positive_prob' array as a CSV file
np.savetxt('yhat_valid_2.txt', yhat_test, delimiter='\n', fmt='%s')


### Image Recognition (PART B)

1.   List item
2.   List item





In [ ]:
import skimage as ski

In [ ]:
from skimage.transform import rotate, rescale, warp, AffineTransform


print(type(train_x[0]))
print(train_x[0].shape)


reshaped_image = np.reshape(train_x[0], (28, 28))
print(reshaped_image.shape)

# Rotions
# Rotate 45
# Rotate 90
# Rotate 180
# Rotate 270
rotated_image = rotate(reshaped_image, 45)
print(rotated_image.shape)

# # Rescale (Blurring)
# # Rescale 2.0 x 0.5
# # Rescale 0.5 x 2.0
# rescaled_image = rescale(reshaped_image, 0.5)
# rescaled_image = rescale(rescaled_image, 2.0)
# print(rescaled_image.shape)


# Translation
# Translate (-2, 0)
# Translate (0, 2)
translation = AffineTransform(translation=(-2, -2))  # Shift 10 pixels left and 10 pixels up
translated_image = warp(reshaped_image, translation)
print(translated_image.shape)

# Horizontal Flip
# Vertical Flip
flipped_image_horizontal = reshaped_image[:, ::-1].copy()
flipped_image_vertical = reshaped_image[::-1, :].copy()
print(flipped_image_horizontal.shape)
print(flipped_image_vertical.shape)



# plt.imshow(reshaped_image, cmap='gray')

<class 'numpy.ndarray'>
(784,)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)


In [ ]:
# plt.imshow(flipped_image_horizontal, cmap='gray')

In [ ]:
# plt.imshow(flipped_image_vertical, cmap='gray')

In [ ]:
# plt.imshow(translated_image, cmap='gray')

In [ ]:
# plt.imshow(rescaled_image, cmap='gray')

In [ ]:
# plt.imshow(rotated_image, cmap='gray')

In [ ]:
import random

def rotate_image(image):
    angle = random.choice([45, 90, 180, 270])
    return rotate(image, angle)

def translate_image(image):
    translation = random.choice([(-2, 0), (0, 2), (3, 3), (-3, 0)])
    transform = AffineTransform(translation=translation)
    return warp(image, transform)

def flip_horizontal(image):
    return image[:, ::-1]

def flip_vertical(image):
    return image[::-1, :]

In [ ]:
augmented_xall_LF_2 = []
augmented_yall_L_2 = []


for i in range(len(modified_x_train)):

    curr_image = modified_x_train[i]
    curr_label = modified_y_train[i]

    # Original image
    reshaped_image = np.reshape(curr_image, (28, 28))
    augmented_xall_LF_2.append(reshaped_image)
    augmented_yall_L_2.append(curr_label)

     # Randomly choose one augmentation technique
    augmentation_function = random.choice([translate_image, flip_horizontal])
    augmented_image = augmentation_function(reshaped_image)

    # Append the augmented image and label
    augmented_xall_LF_2.append(augmented_image)
    augmented_yall_L_2.append(curr_label)

for i in range(len(augmented_xall_LF_2)):
    augmented_xall_LF_2[i] = np.reshape(augmented_xall_LF_2[i], (784, ))

augmented_xall_LF_2 = np.array(augmented_xall_LF_2)
augmented_yall_L_2 = pd.DataFrame(augmented_yall_L_2, columns=['classname'])

In [ ]:
xall_LF_3 = np.vstack([augmented_xall_LF_2, valid_x])
yall_L_3 = np.vstack([augmented_yall_L_2, valid_y_df[['class_name']]])

print(xall_LF_3.shape)
print(yall_L_3.shape)

valid_indicators_L_3 = np.hstack([
    -1 * np.ones(modified_y_train_df.size),
    0 * np.ones(valid_y_df[['class_name']].size)
])

my_splitter_3 = sklearn.model_selection.PredefinedSplit(valid_indicators_L_3)

(10200, 784)
(10200, 1)


### Grid Search

In [ ]:
# Parameter grid
param_grid = {
    'alpha': np.logspace(-2, 2, 4),
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [64, 128]
}

In [ ]:
# Create a mlp model for classification
clf = MLPClassifier(activation='relu', hidden_layer_sizes=100, max_iter=1000, early_stopping = True)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy',
                           cv=my_splitter_3, refit=False, return_train_score=True)

#Run Grid Search
grid_search.fit(augmented_xall_LF_2, augmented_yall_L_2)

/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains c

/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=MLPClassifier(early_stopping=True,
                                     hidden_layer_sizes=100, max_iter=1000),
             param_grid={'alpha': array([1.00000000e-02, 2.15443469e-01, 4.64158883e+00, 1.00000000e+02]),
                         'batch_size': [64, 128],
                         'learning_rate': ['constant', 'adaptive']},
             refit=False, return_train_score=True, scoring='balanced_accuracy')

In [ ]:
clf_3 =  MLPClassifier(activation='relu', hidden_layer_sizes=100, max_iter=1000,
                         alpha=533.6699231206302, batch_size=64, learning_rate='adaptive')

clf_3.fit(augmented_xall_LF_2, augmented_yall_L_2)

yhat_valid_3 = clf_3.predict(valid_x)

bal_acc = sklearn.metrics.balanced_accuracy_score(y_valid, yhat_valid_3)
bal_acc

/Users/tanviamrit/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(alpha=533.6699231206302, batch_size=64, hidden_layer_sizes=100,
              learning_rate='adaptive', max_iter=1000)

In [ ]:
grid_search.best_score_

0.7475

In [ ]:
grid_search.best_params_

{'alpha': 0.0001519911082952933, 'batch_size': 64, 'learning_rate': 'adaptive'}